In [ ]:
import os
import re
import datetime
import subprocess

In [ ]:
rankLibJar = r"E:\IDEAWorkspace\plover\prioritization\ltr\RankLib-2.15.jar"

In [ ]:
project = "Cassandra"
dataFolds = r"E:\IDEAWorkspace\plover\prioritization\ltr\\" + project + r"Data\folds5"

In [ ]:
algos = {
    "0" : "MART", 
    "1" : "RankNet", 
    "2" : "RankBoost", 
    "7" : "ListNet", 
}

In [ ]:
# 1: MethodCalls
# 2: PotentialMethodCall
# 3: Instructions
# 4: PotentialInstruction
# 5: ExecutionFrequency
# 6: MethodFrequency
features = {
#     "1" : "MethodCalls", 
#     "2" : "PotentialMethodCall",
#     "3" : "Instructions", 
#     "4" : "PotentialInstruction",
    "5" : "ExecutionFrequency",
#     "6" : "MethodFrequency"
}

In [ ]:
# cutoff of metrics
cutoffs = ["5", "10", "15", "20", "25", "30"]

In [ ]:
# metrics for test
testMetrics = ["P"]

In [ ]:
cmd = ["java", "-jar", rankLibJar, "-train", "train.data", "-test", "test.data", "-feature", (",").join(features.keys()), 
       "-ranker", "0", "-metric2t", "NDCG@"+cutoffs[0], "-metric2T", "P@"+cutoffs[0]]

In [ ]:
" ".join(cmd)

In [ ]:
show_result = subprocess.check_output(cmd, timeout=30)

In [ ]:
result = show_result.decode(encoding="utf-8", errors="ignore").splitlines()

In [ ]:
train_perf = float(result[-3].split()[-1])
train_perf

In [ ]:
test_perf = float(result[-1].split()[-1])
test_perf

In [ ]:
fileNames = os.listdir(dataFolds)

for algo_id in algos:
#     print("Use ranker " + algos[algo_id] + " for test.")
    
    for metric in testMetrics:
#         print("Use metric=" + metric + " for test.")
        
        for cutoff in cutoffs:
#             print("Use cutoff=" + cutoff + " for test.")
            
            test_perfs = []
            for testFileName in fileNames:
#                 print("Use " + testFileName + " for test.")
                # write test file
                with open("test.data", "w") as outfile:
                    with open(os.path.join(dataFolds, testFileName), "r") as testFile:
                        outfile.write(testFile.read())

                # write train file
                with open("train.data", "w") as outfile:
                    for trainFileName in fileNames:
                        if trainFileName != testFileName:
                            with open(os.path.join(dataFolds, trainFileName), "r") as trainFile:
                                outfile.write(trainFile.read())

                try:
                    # invoke 
                    cmd = ["java", "-jar", rankLibJar, "-train", "train.data", "-test", "test.data", 
                           "-feature", (",").join(features.keys()), "-ranker", algo_id, 
                           "-metric2t", "NDCG@"+cutoff, "-metric2T", metric+"@"+cutoff]
#                            "-save", "Models/"+project+"_"+algos[algo_id]+"_@"+cutoff+"_Test"+testFileName]
#                     print(" ".join(cmd))
                    show_result = subprocess.check_output(cmd)
                    result = show_result.decode(encoding="utf-8", errors="ignore").splitlines()
                    train_perf = float(result[-3].split()[-1])
                    test_perf = float(result[-1].split()[-1])
                    test_perfs.append(test_perf)
#                     print(result[-3])
#                     print(result[-1])
                except subprocess.CalledProcessError as call_err:
                    print(call_err.output.decode(encoding="utf-8", errors="ignore"))
                    
#             print("Average " + metric+"@"+cutoff + " for " + algos[algo_id] + " on test is " + str(sum(test_perfs)/len(test_perfs)))
            print(str(sum(test_perfs)/len(test_perfs)), end="\t")
    print()